# Time Series Forcasting w/ Python

## Content
- Time series analysis
    - ACF plot
    - PACF plot
    - Time seriees decomposition
- Statistical tests
    - ADF test - Test for stationarity
    - Ljung-Box test - Residuals analysis
    - Granger causality - Multivariate forecasting
- Forcasting - Statistical models
    - Moving average model - MA(q)
    - Autoregressive model - AR(q)
    - ARMA(p,q)
    - ARIMA(p,d,q)
    - SARIMA(p,d,q)(P,D,Q)$_m$
    - SARIMAX
    - VARMAX
    - BATS and TBATS
    - Exponential smoothing
- Forcasting - Deep learning models
    - Deep neural network (DNN)
    - Long short-term memory - LSTM
    - Convolutional neural network - CNN
    - Autoregressive deep learning model

## Time series analysis

### ACF plot

The autocorrelation function (ACF) plot shows the autocorrelation coefficients as a function of the lag.

Example use case:
- Use it to determine the order q of a stationary MA(q) process
- A stationary MA(q) process has significant coefficients up until lag q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

# Parameters
n = 500     # Number of observations
q = 3       # MA order

# Generate white noise
white_noise = np.random.standard_normal(n)

# Define MA coefficients
thetas = np.array([1]+list(range(2,2+q)))

# Generate MA(q) time series
ma_series = np.array([
    sum(white_noise[i - j] * thetas[j] for j in range(q+1) if i - j >= 0)
    for i in range(n)
])

# Plot the autocorrelation function (ACF)
plt.figure(figsize=(10, 4))
plot_acf(ma_series, lags=40)
plt.title(f"Autocorrelation Function of MA({q}) Process")
plt.tight_layout()
plt.show()


### PACF plot

The partial autocorrelation function (PACF) plot shows the partial autocorrelation coefficients as a function of the lag.

Example use case:
- Use it to determine the order p of a stationary AR(p) process
- A stationary AR(p) process has significant coefficients up until lag p

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf

# Parameters
n = 500     # Number of observations
p = 3       # AR order

# Generate white noise
white_noise = np.random.standard_normal(n)

# Define AR coefficients
def generate_stable_polynomial(p, radius_min=0.5, radius_max=0.95):
    """
    Generate a polynomial of degree p with all roots inside the unit circle (|z| < 1).
    This ensures no root lies on |z| = 1.
    
    Parameters:
        p (int): Order of the polynomial
        radius_min (float): Minimum modulus of roots (to avoid very small values)
        radius_max (float): Maximum modulus of roots (< 1 to avoid unit circle)

    Returns:
        coeffs (np.ndarray): Polynomial coefficients
        roots (np.ndarray): Roots of the polynomial
    """
    # Generate random complex roots inside the unit circle (|z| < 1)
    angles = np.random.uniform(0, 2 * np.pi, p)
    radii = np.random.uniform(radius_min, radius_max, p)
    roots = radii * np.exp(1j * angles)

    # Get polynomial coefficients from roots
    coeffs = np.poly(roots).real  # Use .real to discard negligible imaginary part due to numerical error

    return coeffs, roots

p = 5
coeffs, roots = generate_stable_polynomial(p)

phis = coeffs

# Initialize time series with zeros (extra p values for warm-up)
ar_series = np.zeros(n)

# Generate AR(p) time series
for i in range(n):
    ar_series[i] = white_noise[i] + sum(ar_series[i - j] * phis[j] if i - j >= 0 else 0 for j in range(p))

# Plot the partial autocorrelation function (PACF)
plt.figure(figsize=(10, 4))
plot_pacf(ar_series)
plt.title(f"Partial Autocorrelation Function of AR({p}) Process")
plt.tight_layout()
plt.show()


### Time series decomposition
Separate the series into 3 components: trend, seasonality, and residuals
- Trend: long-term changes in the series
- Seasonality: periodical variations in the series
- Residuals: what is not explained by trend and seasonality

In [ ]:
import numpy as np
from statsmodels.tsa.seasonal import STL
import matplotlib.pyplot as plt

# m is the frequency of data (i.e., how many observations per season)
m = 5

n = 500

t = np.arange(1, n + 1)

# Generate series
white_noise = np.random.standard_normal(n)
trend = 1 + 0.05 * t
seasonal = 2 * np.cos(np.pi * t / 5) + 3 * np.sin(np.pi * t / 3)

series = trend + seasonal + white_noise

# Perform STL decomposition
decomp = STL(series, period=m).fit()

# Plot the results
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, sharex=True, figsize=(10, 8))

ax1.plot(decomp.observed)
ax1.set_ylabel('Observed')

ax2.plot(decomp.trend)
ax2.set_ylabel('Trend')

ax3.plot(decomp.seasonal)
ax3.set_ylabel('Seasonal')

ax4.plot(decomp.resid)
ax4.set_ylabel('Residuals')

plt.xlabel('Time')
plt.tight_layout()
plt.show()


## Statistical tests

### ADF test – Test for stationarity
A series is stationary it its **mean**, **variance**, and **autocorrelation** are constant over time. Test for stationarity with augmented Dickey-Fuller (ADF) test.
- Null hypothesis: a unit root is present (i.e., the series is not stationary)
- We want a p-value < 0.05

> Notes: 

> The null hypothesis of the Augmented Dickey-Fuller is that there is a unit
root, with the alternative that there is no unit root. If the pvalue is
above a critical size, then we cannot reject that there is a unit root.

> The p-values are obtained through regression surface approximation from
MacKinnon 1994, but using the updated 2010 tables. If the p-value is close
to significant, then the critical values should be used to judge whether
to reject the null.

> The autolag option and maxlag for it are described in Greene.

In [ ]:
import numpy as np
from statsmodels.tsa.stattools import adfuller

# m is the frequency of data (i.e., how many observations per season)
m = 5

n = 500

t = np.arange(1, n + 1)

# Generate series
white_noise = np.random.standard_normal(n)
trend = 1 + 0.05 * t
seasonal = 2 * np.cos(np.pi * t / 5) + 3 * np.sin(np.pi * t / 3)

series = trend + seasonal + white_noise


adf_test_stat, pvalue, usedlag, nobs, crit_val, icbest = adfuller(series)
print(f"adf_test_stat: {adf_test_stat}, \
      \n pvalue: {pvalue}, \
      \n usedlag: {usedlag}, \
      \n nobs: {nobs}, \
      \n crit_val: {crit_val}, \
      \n icbest: {icbest}")

if pvalue < .05:
      print('We have a stationary process')
else: 
      print('We have a non-stationary process')

In [ ]:
import numpy as np
from statsmodels.tsa.stattools import adfuller

# Parameters
n = 500     # Number of observations
q = 3       # MA order

# Generate white noise
white_noise = np.random.standard_normal(n)

# Define MA coefficients (e.g., θ1=2.0, θ2=3.0, θ3=4.0)
thetas = np.array([1]+list(range(2,2+q)))

# Generate MA(q) time series
ma_series = np.array([
    sum(white_noise[i - j] * thetas[j] for j in range(q+1) if i - j >= 0)
    for i in range(n)
])


adf_test_stat, pvalue, usedlag, nobs, crit_val, icbest = adfuller(ma_series)
print(f"adf_test_stat: {adf_test_stat}, \
      \n pvalue: {pvalue}, \
      \n usedlag: {usedlag}, \
      \n nobs: {nobs}, \
      \n crit_val: {crit_val}, \
      \n icbest: {icbest}")

if pvalue < .05:
      print('We have a stationary process')
else: 
      print('We have a non-stationary process')

In [ ]:
import numpy as np
from statsmodels.tsa.stattools import adfuller


# Parameters
n = 500     # Number of observations
p = 3       # AR order

# Generate white noise
white_noise = np.random.standard_normal(n)

# Define AR coefficients
def generate_stable_polynomial(p, radius_min=0.5, radius_max=0.95):
    """
    Generate a polynomial of degree p with all roots inside the unit circle (|z| < 1).
    This ensures no root lies on |z| = 1.
    
    Parameters:
        p (int): Order of the polynomial
        radius_min (float): Minimum modulus of roots (to avoid very small values)
        radius_max (float): Maximum modulus of roots (< 1 to avoid unit circle)

    Returns:
        coeffs (np.ndarray): Polynomial coefficients
        roots (np.ndarray): Roots of the polynomial
    """
    # Generate random complex roots inside the unit circle (|z| < 1)
    angles = np.random.uniform(0, 2 * np.pi, p)
    radii = np.random.uniform(radius_min, radius_max, p)
    roots = radii * np.exp(1j * angles)

    # Get polynomial coefficients from roots
    coeffs = np.poly(roots).real  # Use .real to discard negligible imaginary part due to numerical error

    return coeffs, roots

coeffs, roots = generate_stable_polynomial(p)

phis = coeffs

# Initialize time series with zeros (extra p values for warm-up)
ar_series = np.zeros(n)

# Generate AR(p) time series
for i in range(n):
    ar_series[i] = white_noise[i] + sum(ar_series[i - j] * phis[j] if i - j >= 0 else 0 for j in range(p))

adf_test_stat, pvalue, usedlag, nobs, crit_val, icbest = adfuller(ar_series)
print(f"adf_test_stat: {adf_test_stat}, \
      \n pvalue: {pvalue}, \
      \n usedlag: {usedlag}, \
      \n nobs: {nobs}, \
      \n crit_val: {crit_val}, \
      \n icbest: {icbest}")

if pvalue < .05:
      print('We have a stationary process')
else: 
      print('We have a non-stationary process')



Note: to make a series stationary, use differencing.
- n = 1: difference between consecutive timesteps
- n = 4: difference between values 4 timesteps apart
Differencing removes n data points.

In [ ]:
import numpy as np
from statsmodels.tsa.stattools import adfuller

# m is the frequency of data (i.e., how many observations per season)
m = 5

n = 500

t = np.arange(1, n + 1)

# Generate series
white_noise = np.random.standard_normal(n)
trend = 1 + 0.05 * t
seasonal = 2 * np.cos(np.pi * t / 5) + 3 * np.sin(np.pi * t / 3)

series = trend + seasonal + white_noise

diff_1_series = np.diff(series, n=1)
diff_4_series = np.diff(series, n=4)

adf_test_stat, pvalue, usedlag, nobs, crit_val, icbest = adfuller(diff_1_series)
print(f"diff_1_series\
      \n adf_test_stat: {adf_test_stat}, \
      \n pvalue: {pvalue}, \
      \n usedlag: {usedlag}, \
      \n nobs: {nobs}, \
      \n crit_val: {crit_val}, \
      \n icbest: {icbest}")

if pvalue < .05:
      print('We have a stationary process')
else: 
      print('We have a non-stationary process')


adf_test_stat, pvalue, usedlag, nobs, crit_val, icbest = adfuller(diff_4_series)
print(f"\n diff_4_series\
      \n adf_test_stat: {adf_test_stat}, \
      \n pvalue: {pvalue}, \
      \n usedlag: {usedlag}, \
      \n nobs: {nobs}, \
      \n crit_val: {crit_val}, \
      \n icbest: {icbest}")

if pvalue < .05:
      print('We have a stationary process')
else: 
      print('We have a non-stationary process')

### Ljung-Box test – Residuals analysis
Used to determine if the autocorrelation of a group of data is significantly different from 0. Use it on the residuals to check if they are independent.
- Null hypothesis: the data is independently distributed (i.e., there is no autocorrelation)
- We want a p-value > 0.05

In [ ]:
import numpy as np
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima.model import ARIMA


# Parameters
n = 500  # Number of observations

# Generate white noise
white_noise = np.random.standard_normal(n)

res = ARIMA(white_noise).fit()

# Ljung-Box test
results = acorr_ljungbox(res.resid, lags=[10], return_df=True)

# Interpretation
if results['lb_pvalue'].iloc[-1] > 0.05:
    print("The data appears to be independently distributed (no significant autocorrelation).")
else:
    print("Evidence of autocorrelation found (reject H0 at 5% level).")


In [ ]:
import numpy as np
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima.model import ARIMA


# Parameters
n = 500     # Number of observations
p = 3       # AR order

# Generate white noise
white_noise = np.random.standard_normal(n)

# Define AR coefficients
def generate_stable_polynomial(p, radius_min=0.5, radius_max=0.95):
    """
    Generate a polynomial of degree p with all roots inside the unit circle (|z| < 1).
    This ensures no root lies on |z| = 1.
    
    Parameters:
        p (int): Order of the polynomial
        radius_min (float): Minimum modulus of roots (to avoid very small values)
        radius_max (float): Maximum modulus of roots (< 1 to avoid unit circle)

    Returns:
        coeffs (np.ndarray): Polynomial coefficients
        roots (np.ndarray): Roots of the polynomial
    """
    # Generate random complex roots inside the unit circle (|z| < 1)
    angles = np.random.uniform(0, 2 * np.pi, p)
    radii = np.random.uniform(radius_min, radius_max, p)
    roots = radii * np.exp(1j * angles)

    # Get polynomial coefficients from roots
    coeffs = np.poly(roots).real  # Use .real to discard negligible imaginary part due to numerical error

    return coeffs, roots

coeffs, roots = generate_stable_polynomial(p)

phis = coeffs

# Initialize time series with zeros (extra p values for warm-up)
ar_series = np.zeros(n)

# Generate AR(p) time series
for i in range(n):
    ar_series[i] = white_noise[i] + sum(ar_series[i - j] * phis[j] if i - j >= 0 else 0 for j in range(p))

res = ARIMA(ar_series).fit()

# Ljung-Box test
results = acorr_ljungbox(res.resid, lags=[10], return_df=True)

# Interpretation
if results['lb_pvalue'].iloc[-1] > 0.05:
    print("The data appears to be independently distributed (no significant autocorrelation).")
else:
    print("Evidence of autocorrelation found (reject H0 at 5% level).")


In [ ]:
import numpy as np
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima.model import ARIMA


# Parameters
n = 500     # Number of observations
q = 3       # MA order

# Generate white noise
white_noise = np.random.standard_normal(n)

# Define MA coefficients
thetas = np.array([1]+list(range(2,2+q)))

# Generate MA(q) time series
ma_series = np.array([
    sum(white_noise[i - j] * thetas[j] for j in range(q+1) if i - j >= 0)
    for i in range(n)
])

res = ARIMA(ma_series).fit()

# Ljung-Box test
results = acorr_ljungbox(res.resid, lags=[10], return_df=True)

# Interpretation
if results['lb_pvalue'].iloc[-1] > 0.05:
    print("The data appears to be independently distributed (no significant autocorrelation).")
else:
    print("Evidence of autocorrelation found (reject H0 at 5% level).")

## Forecasting – Statistical models

### Moving average model – MA(q)

The moving average model: the current value depends on the mean of the series, the current error term, and past error terms.
- Denoted as MA(q) where q is the order
- Use ACF plot to find q
- Assumes stationarity. Use only on stationary data

**Equation**
$$
y_t = \mu + \epsilon_t + \theta_1\epsilon_{t-1} + \theta_2\epsilon_{t-2} + \cdots + \theta_q\epsilon_{t-q}
$$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Simulated stationary data: white noise
np.random.seed(42)

# Parameters
n = 500     # Number of observations
q = 5       # MA order

# Generate white noise
white_noise = np.random.standard_normal(n)

# Define MA coefficients
thetas = np.array([1]+list(range(2,2+q)))

# Generate MA(q) time series
ma_series = np.array([
    sum(white_noise[i - j] * thetas[j] for j in range(q+1) if i - j >= 0)
    for i in range(n)
])

# Convert to pandas Series
date_range = pd.date_range(start='2020-01-01', periods=n, freq='d')
series = pd.Series(ma_series, index=date_range)

split = int(series.shape[0] * .96)

# Train-test split
train = series[:split]
test = series[split:]

# Fit MA model (ARIMA with p=0, d=0)
model = ARIMA(train, order=(0, 0, q))  # MA(q) model
results = model.fit()

# Forecast
forecast = results.get_forecast(steps=len(test))
predicted_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

# Plot results
plt.figure(figsize=(10, 5))
plt.plot(series, label='Actual')
plt.plot(predicted_mean, label='Forecast', color='red')
plt.fill_between(predicted_mean.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.axvline(test.index[0], color='gray', linestyle='--', label='Forecast Start')
plt.title('MA Forecast on Stationary Series')
plt.legend()
plt.show()

### Autoregressive model – AR(p)

The autoregressive model is a regression against itself. This means that the present value depends on past values.
- Denoted as AR(p) where p is the order
- Use PACF to find p
- Assumes stationarity. Use only on stationary data

**Equation**
$$
y_t = C + \phi_1y_{t-1} + \phi_2y_{t-2} + \cdots + \phi_qy_{t-q} + \epsilon_t
$$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Simulated stationary data: white noise
np.random.seed(42)

# Parameters
n = 500     # Number of observations
p = 5       # AR order

# Generate white noise
white_noise = np.random.standard_normal(n)

# Define AR coefficients
def generate_stable_polynomial(p, radius_min=0.5, radius_max=0.95):
    """
    Generate a polynomial of degree p with all roots inside the unit circle (|z| < 1).
    This ensures no root lies on |z| = 1.
    
    Parameters:
        p (int): Order of the polynomial
        radius_min (float): Minimum modulus of roots (to avoid very small values)
        radius_max (float): Maximum modulus of roots (< 1 to avoid unit circle)

    Returns:
        coeffs (np.ndarray): Polynomial coefficients
        roots (np.ndarray): Roots of the polynomial
    """
    # Generate random complex roots inside the unit circle (|z| < 1)
    angles = np.random.uniform(0, 2 * np.pi, p)
    radii = np.random.uniform(radius_min, radius_max, p)
    roots = radii * np.exp(1j * angles)

    # Get polynomial coefficients from roots
    coeffs = np.poly(roots).real  # Use .real to discard negligible imaginary part due to numerical error

    return coeffs, roots

coeffs, roots = generate_stable_polynomial(p)

phis = coeffs

# Initialize time series with zeros (extra p values for warm-up)
ar_series = np.zeros(n)

# Generate AR(p) time series
for i in range(n):
    ar_series[i] = white_noise[i] + sum(ar_series[i - j] * phis[j] if i - j >= 0 else 0 for j in range(p))

# Convert to pandas Series
date_range = pd.date_range(start='2020-01-01', periods=n, freq='M')
series = pd.Series(ar_series, index=date_range)

split = int(series.shape[0] * .96)

# Train-test split
train = series[:split]
test = series[split:]

# Fit AR model (ARIMA with q=0, d=0)
model = ARIMA(train, order=(p, 0, 0))  # AR(p) model
results = model.fit()

# Forecast
forecast = results.get_forecast(steps=len(test))
predicted_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

# Plot results
plt.figure(figsize=(10, 5))
plt.plot(series, label='Actual')
plt.plot(predicted_mean, label='Forecast', color='red')
plt.fill_between(predicted_mean.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.axvline(test.index[0], color='gray', linestyle='--', label='Forecast Start')
plt.title('AR Forecast on Stationary Series')
plt.legend()
plt.show()


### ARMA(p,q)

The autoregressive moving average model (ARMA) is the combination of the autoregressive model AR(p), and the moving average model MA(q).
- Denoted as ARMA(p,q) where p is the order of the autoregressive portion, and q is the order of the moving average portion
- Cannot use ACF or PACF to find the order p, and q. Must try different (p,q) value and select the model with the lowest AIC (Akaike’s Information Criterion)
- Assumes stationarity. Use only on stationary data.

**Equation**
$$
y_t = C + \phi_1y_{t-1} + \phi_2y_{t-2} + \cdots + \phi_qy_{t-q} + \theta_1\epsilon_{t-1} + \theta_2\epsilon_{t-2} + \cdots + \theta_q\epsilon_{t-q} + \epsilon_t
$$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Simulated stationary data: noise only
np.random.seed(42)
n_points = 100
data = np.random.normal(0, 1, n_points)

# Convert to pandas Series
date_range = pd.date_range(start='2020-01-01', periods=n_points, freq='M')
series = pd.Series(data, index=date_range)

# Train-test split
train = series[:80]
test = series[80:]

# Fit ARMA model (ARIMA with d=0)
model = ARIMA(train, order=(2, 0, 2))  # ARMA(p,q) => ARIMA(p,0,q)
results = model.fit()

# Forecast
forecast = results.get_forecast(steps=len(test))
predicted_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

# Plot results
plt.figure(figsize=(10, 5))
plt.plot(series, label='Actual')
plt.plot(predicted_mean, label='Forecast', color='red')
plt.fill_between(predicted_mean.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.axvline(test.index[0], color='gray', linestyle='--', label='Forecast Start')
plt.title('ARMA Forecast on Stationary Series')
plt.legend()
plt.show()


### ARIMA(p,d,q)

The autoregressive integrated moving average (ARIMA) model is the combination of the autoregressive model AR(p), and the moving average model MA(q), but in terms of the differenced series.
- Denoted as ARMA(p,d,q), where p is the order of the autoregressive portion, d is the order of integration, and q is the order of the moving average portion
- Can use on non-stationary data 

**Equation**
$$
y_t = C + \phi_1y_{t-1} + \phi_2y_{t-2} + \cdots + \phi_qy_{t-q} + \theta_1\epsilon_{t-1} + \theta_2\epsilon_{t-2} + \cdots + \theta_q\epsilon_{t-q} + \epsilon_t
$$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Simulated non-stationary data: upward trend + noise
np.random.seed(42)
n_points = 100
trend = np.linspace(10, 50, n_points)
noise = np.random.normal(0, 2, n_points)
data = trend + noise

# Convert to pandas Series
date_range = pd.date_range(start='2020-01-01', periods=n_points, freq='M')
series = pd.Series(data, index=date_range)

# Train-test split
train = series[:80]
test = series[80:]

# Fit ARIMA model
model = ARIMA(train, order=(2, 3, 2))  # ARIMA(p,d,q)
results = model.fit()

# Forecast
forecast = results.get_forecast(steps=len(test))
predicted_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

# Plot results
plt.figure(figsize=(10, 5))
plt.plot(series, label='Actual')
plt.plot(predicted_mean, label='Forecast', color='red')
plt.fill_between(predicted_mean.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.axvline(test.index[0], color='gray', linestyle='--', label='Forecast Start')
plt.title('ARIMA Forecast on Non-Stationary Series')
plt.legend()
plt.show()


Note: the order of integration d is simply the number of time a series was differenced to become stationary.

### SARIMA(p,d,q)(P,D,Q)$_m$

The seasonal autoregressive integrated moving average (SARIMA) model includes a seasonal component on top of the ARIMA model.
- Denoted as SARIMA(p,d,q)(P,D,Q)m. Here, p, d, and q have the same meaning as in the ARIMA model.
- P is the seasonal order of the autoregressive portion
- D is the seasonal order of integration
- Q is the seasonal order of the moving average portion
- m is the frequency of the data (i.e., the number of data points in one season)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Simulated monthly sales data with seasonality
np.random.seed(42)
n_months = 120
seasonal_pattern = np.tile([1.0, 1.2, 1.5, 1.3, 1.0, 0.9, 0.8, 0.85, 0.95, 1.1, 1.3, 1.4], n_months // 12)
trend = np.linspace(50, 100, n_months)
noise = np.random.normal(scale=2, size=n_months)

sales = trend * seasonal_pattern + noise
date_index = pd.date_range(start='2015-01-01', periods=n_months, freq='M')
sales_series = pd.Series(sales, index=date_index)

# Train-test split
train = sales_series[:100]
test = sales_series[100:]

# Fit SARIMA model
model = SARIMAX(train,
                order=(1, 1, 1),            # ARIMA part (p,d,q)
                seasonal_order=(1, 1, 1, 12),  # Seasonal part (P,D,Q,m)
                enforce_stationarity=False,
                enforce_invertibility=False)
results = model.fit(disp=False)

# Forecast
forecast = results.get_forecast(steps=len(test))
predicted_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

# Plot
plt.figure(figsize=(10, 5))
plt.plot(sales_series, label='Actual')
plt.plot(predicted_mean, label='Forecast', color='red')
plt.fill_between(predicted_mean.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.axvline(test.index[0], color='gray', linestyle='--', label='Forecast Start')
plt.legend()
plt.title('SARIMA Forecast: Monthly Sales with Yearly Seasonality (m=12)')
plt.show()


### SARIMAX

SARIMAX is the most general model. It combines seasonality, a moving average portion, an autoregressive portion, and exogenous variables.
- Can use external variables to forecast a series

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Simulated data
np.random.seed(42)
n_periods = 100

# Exogenous variable: daily average temperature (with seasonality)
temperature = 10 + 15 * np.sin(np.linspace(0, 3 * np.pi, n_periods)) + np.random.normal(0, 2, n_periods)

# Target variable: energy consumption influenced by temperature
energy = 200 - 3 * temperature + np.random.normal(0, 5, n_periods)

# Convert to pandas Series
date_range = pd.date_range(start='2023-01-01', periods=n_periods, freq='D')
energy_series = pd.Series(energy, index=date_range)
temperature_series = pd.Series(temperature, index=date_range)

# Train-test split
train_size = 80
energy_train = energy_series[:train_size]
energy_test = energy_series[train_size:]
temp_train = temperature_series[:train_size]
temp_test = temperature_series[train_size:]

# Fit SARIMAX model
model = SARIMAX(energy_train,
                exog=temp_train,
                order=(1, 0, 1),        # ARIMA(p,d,q)
                seasonal_order=(1, 0, 1, 7))  # Weekly seasonality
results = model.fit()

# Forecast
forecast = results.get_forecast(steps=len(energy_test), exog=temp_test)
predicted_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

# Plot
plt.figure(figsize=(10, 5))
plt.plot(energy_series, label='Actual')
plt.plot(predicted_mean, label='Forecast', color='red')
plt.fill_between(predicted_mean.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.axvline(energy_test.index[0], color='gray', linestyle='--', label='Forecast Start')
plt.legend()
plt.title('SARIMAX Forecast with Exogenous Variable (Temperature)')
plt.show()


> Caveat: SARIMAX predicts the next timestep. If your horizon is longer than one timestep, then you must forecast your exogenous variables too, which can amplify the error in your model